In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 6.6 MB/s 
     |████████████████████████████████| 596 kB 29.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
     |████████████████████████████████| 895 kB 56.6 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

from transformers import FunnelTokenizerFast, FunnelModel, AdamW, FunnelForSequenceClassification, get_cosine_schedule_with_warmup
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from tqdm.notebook import tqdm

# 데이터 압축해제하기

In [ ]:
import zipfile

PATH = '/content/gdrive/MyDrive/A3_datasets/raw data/'
PATH_2 = '/content/gdrive/MyDrive/A3_datasets/sudo_labeling/'
zip_file = zipfile.ZipFile(PATH + 'data.zip')
zip_file.extractall(PATH_2)

# 데이터 불러오기

In [ ]:
PATH = '/content/gdrive/MyDrive/A3_datasets/sudo_labeling/data/'
dcn = pd.read_csv(PATH + 'discrimination.csv')

In [ ]:
dcn

,text,user_age,user_gender,text_topic,class,age,text_topic_eng
0,내년에 네이버 게임회사 같은데서 사람 엄청 뽑드만. 이럴 줄 알았음 공대를 갈것을 ...,38,female,학력이나 전공에 따라 차별하는 발언,Default,35,Education
1,여자가 아이를 돌보고 남자가 밖에서 돈을 벌어온다' 다 옛날 말이죠. 요즘은 다 반...,33,female,성별 고정관념에 기반한 발언,Default,30,Gender sterotypes
2,"오베라는 남자'라는 책에서 부인의 신체 변화에 따라 남편인 주인공, 오베가 차를 바...",35,female,질병 또는 장애를 비하하는 발언,Default,35,Disabled
3,10년동안 일한 보람이 드디어 생기나보네,37,male,"나이, 사회적 신분, 연차에 의한 차별 발언",Default,35,Age and social status
4,10대들은 모르는 추억의 노래,19,female,"나이, 사회적 신분, 연차에 의한 차별 발언",Default,20,Age and social status
...,...,...,...,...,...,...,...
3765,일본놈들이 하는 짓이 다 그렇지 뭐,30,male,인종에 따른 차별 발언,Spoiled,30,Racism
3766,오잉? 저게 이쁘다는거야? 장님이야뭐야 눈이 어떻게 된거야? 없는거야? 눈어디갔니?...,33,female,질병 또는 장애를 비하하는 발언,Spoiled,30,Disabled
3767,아니 왜 이렇게 태웠어? 팔다리만 길어가지고 배만 툭튀어나온게 까맣게 태우니까 흑인...,33,female,인종에 따른 차별 발언,Spoiled,30,Racism
3768,"한녀들, 김치녀들 취집가려고 한다.",29,female,성별 고정관념에 기반한 발언,Spoiled,25,Gender sterotypes


# 모델 불러오기

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class LoadDataset(Dataset):
    def __init__(self, df, tk):
        self.df = df
        self.tokenizer = tk
        
    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :].values
        # target이 없는경우 (즉, 문장만 입력된 경우)
        if len(row) <= 1:
            text = row[0]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=200,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0].to(device)
            attention_mask = inputs['attention_mask'][0].to(device)

            return input_ids, attention_mask     
            
        # target이 있는 경우 (원래 코드)
        else:
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=200,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0].to(device)
            attention_mask = inputs['attention_mask'][0].to(device)

            return input_ids, attention_mask, y

In [ ]:
model_name_list = ['clean',
                   '개인지칭','기타혐오','성별','성소수자','악플욕설','연령','인종국적','종교','지역'
                   ]
load_md_list = []
PATH = '/content/gdrive/MyDrive/Colab Notebooks/project/'

for md in model_name_list:
    model = FunnelForSequenceClassification.from_pretrained("kykim/funnel-kor-base")
    model.classifier.out_proj =  nn.Sequential( nn.Linear(768, 1),
                                           nn.Sigmoid() )
    try:
        model.load_state_dict(torch.load(PATH + f'{md}.pth')['model_state_dict'])
    except:
        model.load_state_dict(torch.load(PATH + f'{md}.pth'))
    model = model.cuda()
    model = model.eval()
    load_md_list.append(model)

Downloading:   0%|          | 0.00/486 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [ ]:
# check loaded models
len(load_md_list)

1

# 데이터로더 만들기

In [ ]:
tokenizer = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")
discriminator_dataset = LoadDataset(dcn, tk)

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
discriminator_loader = DataLoader(discriminator_dataset, batch_size=6)

# sudo-labeling

In [ ]:
temp_ds = LoadDataset(df, tk)
tmp_loader = DataLoader(temp_ds, batch_size =6)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 개별 모델을 불러옵니다.
md1 = load_md_list[0]
md2 = load_md_list[1]
md3 = load_md_list[2]
md4 = load_md_list[3]
md5 = load_md_list[4]
md6 = load_md_list[5]
md7 = load_md_list[6]
md8 = load_md_list[7]
md9 = load_md_list[8]
md10 = load_md_list[9]

predict_proba_df = pd.DataFrame()

for input_ids_batch, attention_masks_batch, y_batch in tqdm(discriminator_loader):

    # 모델에 입력했을 때, 각 class일 확률 계산
    y_pred1 = md1(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()    
    y_pred2 = md2(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred3 = md3(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred4 = md4(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred5 = md5(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred6 = md6(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred7 = md7(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred8 = md8(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred9 = md9(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    y_pred10 = md10(input_ids_batch, attention_mask=attention_masks_batch)[0].tolist()
    # 예측 확률을 dataframe으로 만듭니다.
    tmp = pd.DataFrame([y_pred1,y_pred2, y_pred3,y_pred4,y_pred5,y_pred6,y_pred7,y_pred8,y_pred9,y_pred10]).T
    predict_proba_df = pd.concat([predict_proba_df, tmp]).reset_index(drop = True)


  0%|          | 0/629 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# evaluate

In [ ]:
predict_proba_df

,0
0,[0.9962190985679626]
1,[0.9737864136695862]
2,[0.9964751601219177]
3,[0.9964710474014282]
4,[0.12007530778646469]
...,...
3765,[0.004801394417881966]
3766,[0.9932507872581482]
3767,[0.002021599095314741]
3768,[0.002626436296850443]


In [ ]:
dcn

,text,user_age,user_gender,text_topic,class,age,text_topic_eng
0,내년에 네이버 게임회사 같은데서 사람 엄청 뽑드만. 이럴 줄 알았음 공대를 갈것을 ...,38,female,학력이나 전공에 따라 차별하는 발언,Default,35,Education
1,여자가 아이를 돌보고 남자가 밖에서 돈을 벌어온다' 다 옛날 말이죠. 요즘은 다 반...,33,female,성별 고정관념에 기반한 발언,Default,30,Gender sterotypes
2,"오베라는 남자'라는 책에서 부인의 신체 변화에 따라 남편인 주인공, 오베가 차를 바...",35,female,질병 또는 장애를 비하하는 발언,Default,35,Disabled
3,10년동안 일한 보람이 드디어 생기나보네,37,male,"나이, 사회적 신분, 연차에 의한 차별 발언",Default,35,Age and social status
4,10대들은 모르는 추억의 노래,19,female,"나이, 사회적 신분, 연차에 의한 차별 발언",Default,20,Age and social status
...,...,...,...,...,...,...,...
3765,일본놈들이 하는 짓이 다 그렇지 뭐,30,male,인종에 따른 차별 발언,Spoiled,30,Racism
3766,오잉? 저게 이쁘다는거야? 장님이야뭐야 눈이 어떻게 된거야? 없는거야? 눈어디갔니?...,33,female,질병 또는 장애를 비하하는 발언,Spoiled,30,Disabled
3767,아니 왜 이렇게 태웠어? 팔다리만 길어가지고 배만 툭튀어나온게 까맣게 태우니까 흑인...,33,female,인종에 따른 차별 발언,Spoiled,30,Racism
3768,"한녀들, 김치녀들 취집가려고 한다.",29,female,성별 고정관념에 기반한 발언,Spoiled,25,Gender sterotypes


In [ ]:
def make_labeled_df(df, df_text_col_name, proba_df, model_name_list = model_name_list):
    # 원본 데이터에서 문장만 가져옵니다.
    df = df[[df_text_col_name]]
    # proba_df의 col_name을 변경합니다.
    proba_col_names = [f'proba_{i}' for i in model_name_list]
    proba_df.columns = proba_col_names
    # proba_df의 datatype을 변경합니다.
    proba_df = proba_df.applymap(lambda x : x[0])
    # 두 데이터를 합칩니다.
    return_df = pd.concat([df, proba_df], axis = 1)
    return return_df

In [ ]:
output_df = make_labeled_df(dcn,'content', predict_proba_df)

In [ ]:
output_df.to_csv('discriminator_labeled.csv', index = False)

,text,pred_proba_clean
4,10대들은 모르는 추억의 노래,[0.12007530778646469]
27,같은 반에 어떤 애가 얼굴에 점이 많아서 점빼는 곳 추천해줌,[0.002895396901294589]
31,갠적으로 머리긴 남자 별론데 왜 연옌이 머리 길면 멋있어 보이지,[0.0023769678082317114]
34,걍 흑인은 힙합 그 자체 인 듯 싶다,[0.004022791050374508]
46,경상도 사람은 억양이 세서 화나있는 거 같은데 또 듣다보면 재밌음,[0.004181030672043562]
...,...,...
3761,깜둥이 새끼들이 여기와서 지랄이냐,[0.002071279101073742]
3763,아주 지랄났다. 어메이징 전라도네. 전라도는 왜 항상 이런식인건지 쯧쯧. 그 지역에...,[0.0024111014790832996]
3765,일본놈들이 하는 짓이 다 그렇지 뭐,[0.004801394417881966]
3767,아니 왜 이렇게 태웠어? 팔다리만 길어가지고 배만 툭튀어나온게 까맣게 태우니까 흑인...,[0.002021599095314741]
